## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων.

Συμπληρώστε το όνομα (NAME) και το AEM σας παρακάτω:

In [11]:
NAME = ""
AEM = ""

**1** Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων breast cancer, χωρίστε το σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70%/30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0). Το σύνολο αφορά τη διάγνωση καρκίνου του μαστού με βάση μεταβλητές που υπολογίζονται από μια ψηφιοποιημένη εικόνα δείγματος μάζας μαστού που λήφθηκε μέσω αναρρόφησης λεπτής βελόνας (FNA). (2 μονάδες)

In [12]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer_data = load_breast_cancer()
X = cancer_data.data
y = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [13]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.1779
assert round(X_test[0][8], 5) == 0.2116

**2** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. (4 μονάδες)

In [14]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        self.estimator = estimator
        self.models = []

    def fit(self, X_train, y_train):
        num_features = X_train.shape[1]
        self.models = []

        for i in range(num_features):
            model = clone(self.estimator)
            X_train_reduced = np.delete(X_train, i, axis=1)
            model.fit(X_train_reduced, y_train)
            self.models.append(model)

    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.models)))

        for i, model in enumerate(self.models):
            X_reduced = np.delete(X, i, axis=1)
            predictions[:, i] = model.predict(X_reduced)

        final_predictions = np.round(np.mean(predictions, axis=1))
        return final_predictions

In [15]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.9006

**3** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. (4 μονάδες)

In [16]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        self.estimator = estimator
        self.n_estimators = n_estimators
        self.models = []
        self.model_weights = []

    def fit(self, X_train, y_train):
        n_samples = X.shape[0]
        # Αρχικοποίηση των βαρών
        sample_weights = np.ones(n_samples) / n_samples

        for _ in range(self.n_estimators):
            # Δημιουργία αντιγράφου του βασικού μοντέλου
            model = clone(self.estimator)
            
            # Εκπαίδευση του μοντέλου με τα τρέχοντα βάρη
            model.fit(X, y, sample_weight=sample_weights)
            
            # Υπολογισμός των προβλέψεων
            y_pred = model.predict(X)
            
            # Υπολογισμός του σφάλματος
            incorrect = (y_pred != y).astype(int)
            error = np.dot(sample_weights, incorrect) / np.sum(sample_weights)
            
            # Υπολογισμός του συντελεστή ενίσχυσης
            alpha = np.log((1 - error) / (error + 1e-10)) / 2
            
            # Ενημέρωση των βαρών των δειγμάτων
            sample_weights *= np.exp(-alpha * y * y_pred)
            sample_weights /= np.sum(sample_weights)
            
            # Αποθήκευση του μοντέλου και του συντελεστή ενίσχυσης
            self.models.append(model)
            self.model_weights.append(alpha)

    def predict(self, X):
        model_preds = np.array([model.predict(X) for model in self.models])
        weighted_preds = np.dot(self.model_weights, model_preds)
        return np.sign(weighted_preds)

In [17]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
print(accuracy_score(ab.predict(X_test), y_test))
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591


0.9239766081871345


AssertionError: 

In [18]:
# Ίδιο αποτέλεσμα και με τη κλάση της sklearn
ab = AdaBoostClassifier(n_estimators=20, algorithm="SAMME", estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591